In [ ]:
# ========================================
# Predicción del Costo Marginal en el SEN
# ========================================

# Importar librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Cargar dataset
df = pd.read_csv("D:\\DOCUMENTOS UNIVERSIDAD\\IA\\proyecto\\cmg_gen_barra.csv")
df.head()

In [ ]:
# Exploración inicial

print('Dimensiones:', df.shape)
print('\nColumnas:', df.columns.tolist())
print('\nValores nulos:')
print(df.isnull().sum())
print('\nResumen estadístico:')
df.describe()

In [ ]:
# Creación de variable objetivo: tipo de generación predominante

df['periodo'] = pd.to_datetime(df['periodo'])
df['timestamp'] = df['periodo'] + pd.to_timedelta(df['dia'] - 1, unit='d') + \
                  pd.to_timedelta(df['hora'], unit='h') + \
                  pd.to_timedelta(df['minuto'], unit='m')

# Agrupamos por barra y timestamp
grupo = df.groupby(['nombre_barra', 'timestamp', 'tipo_gen'])['generacion_kwh'].sum().reset_index()

# Para cada barra y tiempo, seleccionamos el tipo de generación con mayor cantidad de energía generada
predominante = grupo.sort_values(['nombre_barra', 'timestamp', 'generacion_kwh'], ascending=[True, True, False])
predominante = predominante.drop_duplicates(subset=['nombre_barra', 'timestamp'], keep='first')

# Ahora vamos a unir este resultado con los datos de cmg para entrenar el modelo
cmg = df.drop_duplicates(subset=['nombre_barra', 'timestamp'])[['nombre_barra', 'timestamp', 'cmg_peso_kwh', 'hora', 'minuto']]
final_df = pd.merge(predominante, cmg, on=['nombre_barra', 'timestamp'])

In [ ]:
# Preprocesamiento

# Codificar la variable objetivo
le = LabelEncoder()
final_df['tipo_gen_cod'] = le.fit_transform(final_df['tipo_gen'])

# Seleccionamos las variables predictoras
X = final_df[['cmg_peso_kwh', 'hora', 'minuto']]
y = final_df['tipo_gen_cod']

In [ ]:
# Separación de datos

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Usamos RandomForest para empezar
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [ ]:
# Evaluación

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Matriz de Confusión:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title("Matriz de confusión")
plt.show()

In [ ]:
# Visualización de resultados

# Costo marginal real vs predicho
plt.figure(figsize=(12,5))
plt.plot(y_test.values[:200], label="Real")
plt.plot(y_pred[:200], label="Predicción", alpha=0.7)
plt.title("Comparación: Costo Marginal Real vs Predicho")
plt.xlabel("Intervalo de tiempo")
plt.ylabel("CMG ($/kWh)")
plt.legend()
plt.tight_layout()
plt.show()

# Errores absolutos
plt.figure(figsize=(10,4))
plt.hist(abs(y_test - y_pred), bins=50, edgecolor='black')
plt.title("Distribución del Error Absoluto")
plt.xlabel("Error ($/kWh)")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()